In [20]:
import numpy as np
import pandas as pd
import os

import matplotlib
import matplotlib.pyplot as plt
plt.figure(figsize=(15,15))
import seaborn as sns

%matplotlib inline

# 관련 라이브러리 임포트 
import matplotlib.font_manager as fm

#  한글글꼴로 변경
# plt.rcParams['font.family'] = '한글글꼴명'
plt.rcParams['font.size'] = 11.0
# plt.rcParams['font.family'] = 'batang'
plt.rcParams['font.family'] = 'Malgun Gothic'
# 그래프에서 마이너스 폰트 깨지는 문제에 대한 대처
matplotlib.rcParams['axes.unicode_minus'] = False
plt.style.use('ggplot')

![](kc.PNG "LYMNP 로고")
```
우리나라는 국토의 63.2%가 산림으로 이루어져 있습니다. 등산을 해본 경험이 있다면, 산을 오를 수록 그 주위를 이루고 있는 식물, 나무들이 약간씩 다르다는 걸 느낄 수 있습니다. 과연 어떤 조건에 의해서 특정 식물들이 자랄 수 있는 환경이 만들어 진 것일까요? 그걸 알 수 있다면 어떤 지형의 조건이 주어졌을 때, 그곳에 서식하고 있는 숲의 종류를 알 수 있지 않을까요?

이번 컴퍼티션은 콜로라도 주에 있는 산림의 지형적 특징, 흙의 종류, 지표수까지의 위치 등의 정보를 통해 이 지표면에 서식하는 식생(식물)이 어떤 종류인지 예측하는 대회입니다.
```

# 평가 방법

```
이번 컴퍼티션의 평가 방식은 Mean F1-Score입니다. 설명은 아래와 같습니다.

The evaluation metric for this competition is Mean F1-Score. The F1 score, commonly used in information retrieval, measures accuracy using the statistics precision ( \text{p} ) and recall ( \text{r} ).

Precision is the ratio of true positives (\text{tp}) to all predicted positives (\text{tp} + \text{fp}). Recall is the ratio of true positives (\text{tp}) to all actual positives (\text{tp} + \text{fn}). The F1 score is given by:

[ \text{F1} = 2\frac{\text{p} \cdot \text{r}}{\text{p}+\text{r}}\ \ \mathrm{where}\ \ \text{p} = \frac{\text{tp}}{\text{tp}+\text{fp}},\ \ \text{r} = \frac{\text{tp}}{\text{tp}+\text{fn}} ]

The F1 metric weights recall and precision equally, and a good retrieval algorithm will maximize both precision and recall simultaneously. Thus, moderately good performance on both will be favored over extremely good performance on one and poor performance on the other.

The file should contain a header and have the following format:

id,Cover_Type
0,1  
0,2   
0,4   



```

# 데이터
```
Data Fields
Elevation - 높이 (m)
Aspect - 방향 (표면이 향하는 방향)
Slope - 경사도
Horizontal_Distance_To_Hydrology - 지표수까지의 가장 가까운 수평거리
Vertical_Distance_To_Hydrology - 지표수까지의 가장 가까운 수직거리
Horizontal_Distance_To_Roadways - 차도까지의 가장 가까운 수평거리
Horizontal_Distance_To_Fire_Points - 불이 났던 지점으로부터 가장 가까운 수평거리
Hillshade_9am - 하지점(Summer solstice) 기준으로 9am의 태양의 위치에 따른 음영
Hillshade_Noon - 하지점(Summer solstice) 기준으로 12am의 태양의 위치에 따른 음영
Hillshade_3pm - 하지점(Summer solstice) 기준으로 3pm의 태양의 위치에 따른 음영
Wilderness_Area - 야생 지역 위치 (아래 참고)
Soil_Type - 토양 타입 (아래 참고)
Cover_Type - 산림 타입 (Target) (아래 참고)
Forest Cover Type 별 설명
Spruce/Fir wiki
Lodgepole Pine wiki
Ponderosa Pine wiki
Krummholz wiki
Wilderness Area Type 별 설명 (데이터는 0 ~ 3로 표기)
Rawah Wilderness Area
Neota Wilderness Area
Comanche Peak Wilderness Area
Cache la Poudre Wilderness Area
Soil Type 별 설명 (데이터는 0 ~ 39로 표기 )
1 Cathedral family - Rock outcrop complex, extremely stony.
2 Vanet - Ratake families complex, very stony.
3 Haploborolis - Rock outcrop complex, rubbly.
4 Ratake family - Rock outcrop complex, rubbly.
5 Vanet family - Rock outcrop complex complex, rubbly.
6 Vanet - Wetmore families - Rock outcrop complex, stony.
7 Gothic family.
8 Supervisor - Limber families complex.
9 Troutville family, very stony.
10 Bullwark - Catamount families - Rock outcrop complex, rubbly.
11 Bullwark - Catamount families - Rock land complex, rubbly.
12 Legault family - Rock land complex, stony.
13 Catamount family - Rock land - Bullwark family complex, rubbly.
14 Pachic Argiborolis - Aquolis complex.
15 unspecified in the USFS Soil and ELU Survey.
16 Cryaquolis - Cryoborolis complex.
17 Gateview family - Cryaquolis complex.
18 Rogert family, very stony.
19 Typic Cryaquolis - Borohemists complex.
20 Typic Cryaquepts - Typic Cryaquolls complex.
21 Typic Cryaquolls - Leighcan family, till substratum complex.
22 Leighcan family, till substratum, extremely bouldery.
23 Leighcan family, till substratum - Typic Cryaquolls complex.
24 Leighcan family, extremely stony.
25 Leighcan family, warm, extremely stony.
26 Granile - Catamount families complex, very stony.
27 Leighcan family, warm - Rock outcrop complex, extremely stony.
28 Leighcan family - Rock outcrop complex, extremely stony.
29 Como - Legault families complex, extremely stony.
30 Como family - Rock land - Legault family complex, extremely stony.
31 Leighcan - Catamount families complex, extremely stony.
32 Catamount family - Rock outcrop - Leighcan family complex, extremely stony.
33 Leighcan - Catamount families - Rock outcrop complex, extremely stony.
34 Cryorthents - Rock land complex, extremely stony.
35 Cryumbrepts - Rock outcrop - Cryaquepts complex.
36 Bross family - Rock land - Cryumbrepts complex, extremely stony.
37 Rock outcrop - Cryumbrepts - Cryorthents complex, extremely stony.
38 Leighcan - Moran families - Cryaquolls complex, extremely stony.
39 Moran family - Cryorthents - Leighcan family complex, extremely stony.
40 Moran family - Cryorthents - Rock land complex, extremely stony.
```



# 파일
```
train.csv - 훈련 셋
test.csv - 테스트 셋
sample_submission.csv - 제출 파일 포맷
```

# 데이터 로드

In [21]:
train = pd.read_csv('soil/train.csv' ) 
test = pd.read_csv('soil/test.csv' ) 
train

,id,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Soil_Type,Wilderness_Area,Cover_Type
0,0,2983,339,23,97.0,NaN,1441.0,223.0,205.0,105.0,3487.0,21,0,1
1,1,3046,375,5,69.0,-3.0,865.0,223.0,225.0,149.0,1522.0,0,2,2
2,2,2540,199,10,128.0,2.0,1009.0,216.0,245.0,146.0,1977.0,0,0,2
3,3,2893,351,21,54.0,19.0,4886.0,247.0,195.0,127.0,1777.0,18,2,2
4,4,2529,58,35,92.0,42.0,455.0,225.0,249.0,168.0,264.0,25,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
826219,826219,2626,144,17,60.0,-3.0,1001.0,220.0,257.0,177.0,369.0,25,0,2
826220,826220,3147,47,27,38.0,24.0,1393.0,204.0,224.0,68.0,797.0,0,2,1
826221,826221,3362,203,8,506.0,0.0,802.0,182.0,220.0,116.0,618.0,0,1,1
826222,826222,3127,271,18,127.0,133.0,NaN,192.0,228.0,147.0,NaN,27,2,1


In [22]:
# y 설정
y = train['Cover_Type']
# y_test = test['Cover_Type']
del(train['Cover_Type'])

In [23]:
# 쓸모없는 인덱스 제거
test.drop('id',axis=1 , inplace=True)
train.drop('id',axis=1 , inplace=True)

In [24]:
# 카테고리컬 컬럼, 수치형 컬럼 분류
cat_columns =  ['Soil_Type' ,'Wilderness_Area']
num_columns = [c for c    in train.columns if (c not in cat_columns)]
print('Categorical Columns: \n{}\n'.format(cat_columns))
print('Numeric Columns: \n{}'.format(num_columns))

Categorical Columns: 
['Soil_Type', 'Wilderness_Area']

Numeric Columns: 
['Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology', 'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways', 'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm', 'Horizontal_Distance_To_Fire_Points']


In [25]:
from sklearn.preprocessing import MinMaxScaler
# 스케일링

scaler = MinMaxScaler()
scaler2 = MinMaxScaler()

scaler.fit(train[num_columns])
scaler2.fit(test[['Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology', 'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways', 'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm', 'Horizontal_Distance_To_Fire_Points']])

MinMaxScaler()

In [26]:
scaler.transform(train[num_columns])

array([[0.47241923, 0.84862385, 0.39393939, ..., 0.66816143, 0.48447205,
        0.45376526],
       [0.49724192, 0.93119266, 0.12121212, ..., 0.75784753, 0.62111801,
        0.21624562],
       [0.29787234, 0.52752294, 0.1969697 , ..., 0.84753363, 0.61180124,
        0.27124381],
       ...,
       [0.62174941, 0.53669725, 0.16666667, ..., 0.73542601, 0.51863354,
        0.1069745 ],
       [0.52915682, 0.69266055, 0.31818182, ..., 0.77130045, 0.61490683,
               nan],
       [0.55200946, 0.11697248, 0.3030303 , ..., 0.73542601, 0.54037267,
        0.58588178]])

In [27]:
scaler2.fit_transform(test[['Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology', 'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways', 'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm', 'Horizontal_Distance_To_Fire_Points']])

array([[0.51432998, 0.18202765, 0.53125   , ..., 0.80365297, 0.43302181,
        0.19639181],
       [0.3140976 , 0.47926267, 0.328125  , ..., 0.82191781, 0.52024922,
        0.18125681],
       [0.66924864, 0.16820276, 0.03125   , ..., 0.74885845, 0.73208723,
        0.53275215],
       ...,
       [0.3954299 , 0.57142857, 0.46875   , ..., 0.83105023, 0.65732087,
        0.06029786],
       [0.41169636, 0.48617512, 0.328125  , ..., 0.75342466, 0.73208723,
        0.14711224],
       [0.58326878, 0.30645161, 0.15625   , ..., 0.84474886, 0.5576324 ,
        0.35452234]])

In [28]:
scaled_data = pd.DataFrame(scaler.transform(train[num_columns]), columns=num_columns)
scaled_data2 = pd.DataFrame(scaler2.transform(test[['Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology',
 'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways',
 'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm', 'Horizontal_Distance_To_Fire_Points']]) ,columns=['Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology',
 'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways',
 'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm', 'Horizontal_Distance_To_Fire_Points'])

In [29]:
from sklearn.model_selection import train_test_split
# Train Test 분리
X_train, X_validation, y_train, y_validation = train_test_split(scaled_data, y, train_size=0.75, random_state=42)

In [30]:
from sklearn.model_selection import train_test_split
# Train Test 분리
X_train, X_validation, y_train, y_validation = train_test_split(train, y, train_size=0.75, random_state=42)

# 캣 부스트 사용
```
GBM의 치명적인 문제점 중 하나로 과적합 문제가 존재한다. 
CatBoost는 이 과적합 문제를 해결하면서 동시에 기존 GBM 계열 알고리즘인 
XGBoost, LightGBM 알고리즘보다 학습 속도를 개선하는 장점
 결측치가 매우 많은 데이터셋에는 부적합한 모델


```

In [31]:
from catboost import CatBoostClassifier, Pool, cv



In [32]:
# 데이터 확인
X_train, X_validation, y_train, y_validation

(        Elevation  Aspect  Slope  Horizontal_Distance_To_Hydrology  \
 457026       2749      46     41                             581.0   
 647971       3197      68     12                             102.0   
 540916       2810     380      7                             293.0   
 479225       2779     188      4                             209.0   
 716814       2780      60      7                             118.0   
 ...           ...     ...    ...                               ...   
 259178       3366       2      7                             666.0   
 365838       3216     181     11                              36.0   
 131932       2590     130     22                             595.0   
 671155       3332     276     10                              54.0   
 121958       2687      24      9                             196.0   
 
         Vertical_Distance_To_Hydrology  Horizontal_Distance_To_Roadways  \
 457026                            40.0                            280

In [33]:
# y train 인코딩
pd.get_dummies( y_train)

,1,2,3,4
457026,0,1,0,0
647971,1,0,0,0
540916,0,1,0,0
479225,0,1,0,0
716814,0,1,0,0
...,...,...,...,...
259178,1,0,0,0
365838,1,0,0,0
131932,0,0,1,0
671155,1,0,0,0


In [38]:
from sklearn.metrics import accuracy_score, log_loss, make_scorer
from sklearn.model_selection import GridSearchCV

In [35]:
import catboost as cb
cb_dtrain = cb.Pool(data = X_train, label = y_train) # 학습 데이터를 Catboost 모델에 맞게 변환
cb_param = {'max_depth': 10, # 트리 깊이
            'learning_rate': 0.1, # Step Size
            'n_estimators': 100, # Number of trees, 트리 생성 개수
            'eval_metric': 'Accuracy', # 평가 척도
            'task_type' : 'GPU',
            'loss_function': 'MultiClass'} # 손실 함수, 목적 함수
cb_model = cb.train(pool = cb_dtrain, params = cb_param ) # 학습 진행
cb_model_predict = np.argmax(cb_model.predict(X_validation), axis = 1) + 1 # 평가 데이터 예측, Softmax의 결과값 중 가장 큰 값의 Label로 예측, 인덱스의 순서를 맞추기 위해 +1
print("Accuracy: %.4f" % (accuracy_score(y_validation, cb_model_predict) * 100), "%") # 정확도 % 계산

0:	learn: 0.8841186	total: 45.5ms	remaining: 4.5s
1:	learn: 0.8849771	total: 80.9ms	remaining: 3.96s
2:	learn: 0.8911772	total: 114ms	remaining: 3.67s
3:	learn: 0.8953827	total: 146ms	remaining: 3.5s
4:	learn: 0.8966834	total: 178ms	remaining: 3.39s
5:	learn: 0.8976807	total: 210ms	remaining: 3.29s
6:	learn: 0.8984537	total: 243ms	remaining: 3.23s
7:	learn: 0.8975161	total: 275ms	remaining: 3.17s
8:	learn: 0.8978792	total: 307ms	remaining: 3.11s
9:	learn: 0.8979696	total: 337ms	remaining: 3.04s
10:	learn: 0.8990766	total: 373ms	remaining: 3.02s
11:	learn: 0.9000400	total: 407ms	remaining: 2.98s
12:	learn: 0.9009970	total: 442ms	remaining: 2.96s
13:	learn: 0.9021153	total: 480ms	remaining: 2.95s
14:	learn: 0.9034644	total: 531ms	remaining: 3.01s
15:	learn: 0.9036468	total: 576ms	remaining: 3.02s
16:	learn: 0.9043891	total: 612ms	remaining: 2.98s
17:	learn: 0.9048781	total: 647ms	remaining: 2.95s
18:	learn: 0.9057415	total: 687ms	remaining: 2.93s
19:	learn: 0.9067840	total: 732ms	remaini

파라미터를 임의로 설정 => 정확도 92.7521%
 
 
 정확도 개선 위해 그리드 서치 진행

In [43]:
from sklearn.model_selection import GridSearchCV
import catboost as cb
from sklearn.model_selection import cross_val_score

cb = CatBoostClassifier(early_stopping_rounds=100) # earlystopping_round =100, use_best_model = True 
cb_param = {'max_depth': [8,9], # 트리 깊이
            'learning_rate': [0.25,0.45 ], # Step Size
            'n_estimators': [300],# Number of trees, 트리 생성 개수
            'eval_metric': ['Accuracy'], # 평가 척도
            'task_type' : ['GPU'],
            'loss_function': ['MultiClass']  
           }
grid_cb= GridSearchCV(estimator = cb, param_grid=cb_param)

grid_cb.fit(X_train,y_train)

Custom logger is already specified. Specify more than one logger at same time is not thread safe.

0:	learn: 0.8745940	total: 21.9ms	remaining: 6.55s
1:	learn: 0.8825156	total: 40.6ms	remaining: 6.04s
2:	learn: 0.8858501	total: 59ms	remaining: 5.84s
3:	learn: 0.8877362	total: 76.8ms	remaining: 5.68s
4:	learn: 0.8922507	total: 93.9ms	remaining: 5.54s
5:	learn: 0.8949376	total: 113ms	remaining: 5.54s
6:	learn: 0.8987501	total: 132ms	remaining: 5.52s
7:	learn: 0.9008944	total: 150ms	remaining: 5.47s
8:	learn: 0.9025304	total: 170ms	remaining: 5.49s
9:	learn: 0.9040151	total: 189ms	remaining: 5.49s
10:	learn: 0.9055643	total: 208ms	remaining: 5.46s
11:	learn: 0.9072022	total: 227ms	remaining: 5.44s
12:	learn: 0.9091489	total: 246ms	remaining: 5.43s
13:	learn: 0.9103229	total: 264ms	remaining: 5.4s
14:	learn: 0.9119588	total: 282ms	remaining: 5.36s
15:	learn: 0.9131792	total: 300ms	remaining: 5.33s
16:	learn: 0.9138288	total: 319ms	remaining: 5.32s
17:	learn: 0.9144521	total: 336ms	remaining: 5.26s
18:	learn: 0.9153155	total: 355ms	remaining: 5.25s
19:	learn: 0.9163039	total: 373ms	remai

167:	learn: 0.9362037	total: 3.1s	remaining: 2.44s
168:	learn: 0.9362844	total: 3.12s	remaining: 2.42s
169:	learn: 0.9362965	total: 3.14s	remaining: 2.4s
170:	learn: 0.9363227	total: 3.15s	remaining: 2.38s
171:	learn: 0.9363610	total: 3.18s	remaining: 2.36s
172:	learn: 0.9364034	total: 3.19s	remaining: 2.35s
173:	learn: 0.9364760	total: 3.21s	remaining: 2.33s
174:	learn: 0.9365063	total: 3.23s	remaining: 2.31s
175:	learn: 0.9365406	total: 3.25s	remaining: 2.29s
176:	learn: 0.9365708	total: 3.27s	remaining: 2.27s
177:	learn: 0.9366919	total: 3.29s	remaining: 2.25s
178:	learn: 0.9366555	total: 3.3s	remaining: 2.23s
179:	learn: 0.9367040	total: 3.32s	remaining: 2.21s
180:	learn: 0.9367221	total: 3.34s	remaining: 2.19s
181:	learn: 0.9368230	total: 3.36s	remaining: 2.18s
182:	learn: 0.9368815	total: 3.38s	remaining: 2.16s
183:	learn: 0.9369884	total: 3.39s	remaining: 2.14s
184:	learn: 0.9370408	total: 3.42s	remaining: 2.12s
185:	learn: 0.9370449	total: 3.43s	remaining: 2.1s
186:	learn: 0.93

34:	learn: 0.9234347	total: 634ms	remaining: 4.8s
35:	learn: 0.9236587	total: 651ms	remaining: 4.78s
36:	learn: 0.9240601	total: 670ms	remaining: 4.76s
37:	learn: 0.9242416	total: 689ms	remaining: 4.75s
38:	learn: 0.9245402	total: 709ms	remaining: 4.74s
39:	learn: 0.9247883	total: 727ms	remaining: 4.73s
40:	learn: 0.9250848	total: 749ms	remaining: 4.73s
41:	learn: 0.9252926	total: 766ms	remaining: 4.71s
42:	learn: 0.9255710	total: 785ms	remaining: 4.69s
43:	learn: 0.9256940	total: 803ms	remaining: 4.67s
44:	learn: 0.9257566	total: 822ms	remaining: 4.66s
45:	learn: 0.9259421	total: 840ms	remaining: 4.64s
46:	learn: 0.9262952	total: 859ms	remaining: 4.62s
47:	learn: 0.9264444	total: 876ms	remaining: 4.6s
48:	learn: 0.9267430	total: 894ms	remaining: 4.58s
49:	learn: 0.9269507	total: 912ms	remaining: 4.56s
50:	learn: 0.9269709	total: 930ms	remaining: 4.54s
51:	learn: 0.9270798	total: 948ms	remaining: 4.52s
52:	learn: 0.9272392	total: 967ms	remaining: 4.5s
53:	learn: 0.9274369	total: 988ms	

200:	learn: 0.9376984	total: 3.68s	remaining: 1.81s
201:	learn: 0.9377126	total: 3.7s	remaining: 1.79s
202:	learn: 0.9377388	total: 3.72s	remaining: 1.78s
203:	learn: 0.9377650	total: 3.74s	remaining: 1.76s
204:	learn: 0.9378316	total: 3.76s	remaining: 1.74s
205:	learn: 0.9379607	total: 3.78s	remaining: 1.72s
206:	learn: 0.9379930	total: 3.79s	remaining: 1.71s
207:	learn: 0.9380575	total: 3.81s	remaining: 1.69s
208:	learn: 0.9381241	total: 3.83s	remaining: 1.67s
209:	learn: 0.9381967	total: 3.85s	remaining: 1.65s
210:	learn: 0.9382350	total: 3.87s	remaining: 1.63s
211:	learn: 0.9382814	total: 3.88s	remaining: 1.61s
212:	learn: 0.9382935	total: 3.9s	remaining: 1.59s
213:	learn: 0.9383823	total: 3.92s	remaining: 1.58s
214:	learn: 0.9384045	total: 3.94s	remaining: 1.56s
215:	learn: 0.9385820	total: 3.96s	remaining: 1.54s
216:	learn: 0.9386163	total: 3.98s	remaining: 1.52s
217:	learn: 0.9386324	total: 4s	remaining: 1.5s
218:	learn: 0.9387252	total: 4.02s	remaining: 1.49s
219:	learn: 0.9387

...

54:	learn: 0.9331114	total: 1.3s	remaining: 5.8s
55:	learn: 0.9332022	total: 1.32s	remaining: 5.77s
56:	learn: 0.9334302	total: 1.35s	remaining: 5.76s
57:	learn: 0.9338316	total: 1.38s	remaining: 5.76s
58:	learn: 0.9339748	total: 1.4s	remaining: 5.73s
59:	learn: 0.9341079	total: 1.43s	remaining: 5.71s
60:	learn: 0.9342088	total: 1.45s	remaining: 5.68s
61:	learn: 0.9343278	total: 1.47s	remaining: 5.66s
62:	learn: 0.9345457	total: 1.5s	remaining: 5.63s
63:	learn: 0.9348019	total: 1.52s	remaining: 5.62s
64:	learn: 0.9350258	total: 1.55s	remaining: 5.6s
65:	learn: 0.9351125	total: 1.57s	remaining: 5.58s
66:	learn: 0.9351589	total: 1.6s	remaining: 5.56s
67:	learn: 0.9352638	total: 1.62s	remaining: 5.54s
68:	learn: 0.9353082	total: 1.64s	remaining: 5.5s
69:	learn: 0.9355301	total: 1.67s	remaining: 5.48s
70:	learn: 0.9357237	total: 1.69s	remaining: 5.46s
71:	learn: 0.9358185	total: 1.72s	remaining: 5.43s
72:	learn: 0.9359436	total: 1.74s	remaining: 5.41s
73:	learn: 0.9360223	total: 1.76s	rema

219:	learn: 0.9529809	total: 5.37s	remaining: 1.95s
220:	learn: 0.9531302	total: 5.4s	remaining: 1.93s
221:	learn: 0.9531968	total: 5.42s	remaining: 1.91s
222:	learn: 0.9532976	total: 5.45s	remaining: 1.88s
223:	learn: 0.9533581	total: 5.47s	remaining: 1.86s
224:	learn: 0.9534287	total: 5.5s	remaining: 1.83s
225:	learn: 0.9535942	total: 5.53s	remaining: 1.81s
226:	learn: 0.9536950	total: 5.55s	remaining: 1.78s
227:	learn: 0.9537515	total: 5.58s	remaining: 1.76s
228:	learn: 0.9537454	total: 5.6s	remaining: 1.74s
229:	learn: 0.9539310	total: 5.62s	remaining: 1.71s
230:	learn: 0.9539835	total: 5.64s	remaining: 1.69s
231:	learn: 0.9541045	total: 5.67s	remaining: 1.66s
232:	learn: 0.9541630	total: 5.69s	remaining: 1.64s
233:	learn: 0.9542417	total: 5.72s	remaining: 1.61s
234:	learn: 0.9543526	total: 5.74s	remaining: 1.59s
235:	learn: 0.9545039	total: 5.76s	remaining: 1.56s
236:	learn: 0.9546229	total: 5.79s	remaining: 1.54s
237:	learn: 0.9546229	total: 5.82s	remaining: 1.51s
238:	learn: 0.9

88:	learn: 0.9310744	total: 1.74s	remaining: 4.13s
89:	learn: 0.9311244	total: 1.76s	remaining: 4.11s
90:	learn: 0.9311405	total: 1.78s	remaining: 4.09s
91:	learn: 0.9312680	total: 1.8s	remaining: 4.07s
92:	learn: 0.9313245	total: 1.82s	remaining: 4.05s
93:	learn: 0.9314068	total: 1.84s	remaining: 4.04s
94:	learn: 0.9314294	total: 1.86s	remaining: 4.01s
95:	learn: 0.9315101	total: 1.88s	remaining: 3.99s
96:	learn: 0.9315650	total: 1.9s	remaining: 3.97s
97:	learn: 0.9315892	total: 1.92s	remaining: 3.95s
98:	learn: 0.9316844	total: 1.94s	remaining: 3.93s
99:	learn: 0.9318506	total: 1.96s	remaining: 3.91s
100:	learn: 0.9318651	total: 1.97s	remaining: 3.89s
101:	learn: 0.9319523	total: 1.99s	remaining: 3.87s
102:	learn: 0.9319603	total: 2.01s	remaining: 3.85s
103:	learn: 0.9320055	total: 2.03s	remaining: 3.83s
104:	learn: 0.9320426	total: 2.05s	remaining: 3.8s
105:	learn: 0.9321379	total: 2.07s	remaining: 3.78s
106:	learn: 0.9321766	total: 2.08s	remaining: 3.76s
107:	learn: 0.9322621	total

253:	learn: 0.9388560	total: 5s	remaining: 906ms
254:	learn: 0.9388915	total: 5.02s	remaining: 886ms
255:	learn: 0.9389496	total: 5.04s	remaining: 866ms
256:	learn: 0.9389705	total: 5.06s	remaining: 847ms
257:	learn: 0.9389818	total: 5.08s	remaining: 827ms
258:	learn: 0.9390125	total: 5.1s	remaining: 808ms
259:	learn: 0.9390222	total: 5.12s	remaining: 788ms
260:	learn: 0.9390512	total: 5.14s	remaining: 768ms
261:	learn: 0.9390674	total: 5.16s	remaining: 749ms
262:	learn: 0.9391384	total: 5.18s	remaining: 729ms
263:	learn: 0.9392562	total: 5.2s	remaining: 710ms
264:	learn: 0.9392998	total: 5.22s	remaining: 690ms
265:	learn: 0.9393207	total: 5.24s	remaining: 670ms
266:	learn: 0.9393320	total: 5.26s	remaining: 650ms
267:	learn: 0.9393772	total: 5.28s	remaining: 631ms
268:	learn: 0.9394063	total: 5.3s	remaining: 611ms
269:	learn: 0.9393643	total: 5.32s	remaining: 591ms
270:	learn: 0.9394531	total: 5.34s	remaining: 572ms
271:	learn: 0.9395241	total: 5.36s	remaining: 552ms
272:	learn: 0.9395

GridSearchCV(estimator=<catboost.core.CatBoostClassifier object at 0x0000020FA2C6E8E0>,
             param_grid={'eval_metric': ['Accuracy'],
                         'learning_rate': [0.25, 0.45],
                         'loss_function': ['MultiClass'], 'max_depth': [8, 9],
                         'n_estimators': [300], 'task_type': ['GPU']})

In [44]:
# 최적의 파라미터
grid_cb.best_params_ 

{'eval_metric': 'Accuracy',
 'learning_rate': 0.25,
 'loss_function': 'MultiClass',
 'max_depth': 8,
 'n_estimators': 300,
 'task_type': 'GPU'}

## 최적 파라미터
```
 'learning_rate': 0.25,
 'loss_function': 'MultiClass',
 'max_depth': 8,
 'n_estimators': 300,

```

In [36]:
# 
cb_dtrain = cb.Pool(data = X_train, label = y_train) # 학습 데이터를 Catboost 모델에 맞게 변환
cb_param = {'eval_metric': 'Accuracy',
 'learning_rate': 0.25,
 'loss_function': 'MultiClass',
 'max_depth': 8,
 'n_estimators': 300,
 'task_type': 'GPU'}
cb_model = cb.train(pool = cb_dtrain, params = cb_param ) # 학습 진행
cb_model_predict = np.argmax(cb_model.predict(X_validation), axis = 1) + 1 # 평가 데이터 예측, Softmax의 결과값 중 가장 큰 값의 Label로 예측, 인덱스의 순서를 맞추기 위해 +1
print("Accuracy: %.4f" % (accuracy_score(y_validation, cb_model_predict) * 100), "%") # 정확도 % 계산

0:	learn: 0.8744199	total: 36.1ms	remaining: 10.8s
1:	learn: 0.8808959	total: 57.6ms	remaining: 8.58s
2:	learn: 0.8842009	total: 76.9ms	remaining: 7.62s
3:	learn: 0.8870072	total: 96.5ms	remaining: 7.14s
4:	learn: 0.8926248	total: 115ms	remaining: 6.8s
5:	learn: 0.8961153	total: 133ms	remaining: 6.54s
6:	learn: 0.8995527	total: 153ms	remaining: 6.42s
7:	learn: 0.9019991	total: 172ms	remaining: 6.29s
8:	learn: 0.9031723	total: 194ms	remaining: 6.26s
9:	learn: 0.9051444	total: 214ms	remaining: 6.21s
10:	learn: 0.9054445	total: 234ms	remaining: 6.15s
11:	learn: 0.9069082	total: 254ms	remaining: 6.08s
12:	learn: 0.9088157	total: 275ms	remaining: 6.08s
13:	learn: 0.9100196	total: 294ms	remaining: 6.01s
14:	learn: 0.9105344	total: 314ms	remaining: 5.96s
15:	learn: 0.9118786	total: 333ms	remaining: 5.92s
16:	learn: 0.9133504	total: 351ms	remaining: 5.85s
17:	learn: 0.9140249	total: 369ms	remaining: 5.78s
18:	learn: 0.9151739	total: 389ms	remaining: 5.75s
19:	learn: 0.9160793	total: 410ms	rema

164:	learn: 0.9352686	total: 3.37s	remaining: 2.76s
165:	learn: 0.9353299	total: 3.39s	remaining: 2.74s
166:	learn: 0.9353670	total: 3.41s	remaining: 2.72s
167:	learn: 0.9354299	total: 3.43s	remaining: 2.69s
168:	learn: 0.9354203	total: 3.45s	remaining: 2.67s
169:	learn: 0.9354154	total: 3.47s	remaining: 2.65s
170:	learn: 0.9355268	total: 3.49s	remaining: 2.63s
171:	learn: 0.9357446	total: 3.51s	remaining: 2.61s
172:	learn: 0.9357462	total: 3.53s	remaining: 2.59s
173:	learn: 0.9356994	total: 3.55s	remaining: 2.57s
174:	learn: 0.9357269	total: 3.57s	remaining: 2.55s
175:	learn: 0.9357882	total: 3.59s	remaining: 2.53s
176:	learn: 0.9357850	total: 3.6s	remaining: 2.5s
177:	learn: 0.9358221	total: 3.62s	remaining: 2.48s
178:	learn: 0.9358818	total: 3.64s	remaining: 2.46s
179:	learn: 0.9359576	total: 3.66s	remaining: 2.44s
180:	learn: 0.9359802	total: 3.68s	remaining: 2.42s
181:	learn: 0.9359818	total: 3.7s	remaining: 2.4s
182:	learn: 0.9360141	total: 3.72s	remaining: 2.38s
183:	learn: 0.93

# 정확도 = 93.151%

# Submission 제출

In [47]:
submission = pd.read_csv("soil/sample_submission.csv")
submission['Cover_Type'] = np.argmax(cb_model.predict(test), axis = 1) + 1
submission.to_csv("PDH_submission.csv",index=False)

# 확인

In [48]:
pd.read_csv("soil/sample_submission.csv")

,id,Cover_Type
0,0,-1
1,1,-1
2,2,-1
3,3,-1
4,4,-1
...,...,...
550811,550811,-1
550812,550812,-1
550813,550813,-1
550814,550814,-1


# 내 순위
![MY RANK](9th.PNG "개인 순위") 

![MY RANK2](11th.PNG "public 순위") 